### Preparazione del dataset con conversione delle label per YOLO e conservazione sul Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import os
import csv
from PIL import Image
from tqdm.notebook import tqdm

import shutil
import random
from pathlib import Path

In [2]:
# Copia l'archivio sul disco della macchina
shutil.copy('/content/drive/MyDrive/Simone/mio_dataset.tar', '/content/mio-dataset.tar')

'/content/mio-dataset.tar'

In [3]:
# Ci mette circa 40 secondi
!tar -xf "mio-dataset.tar" -C ./

Output streaming troncato alle ultime 5000 righe.
MIO-TCD-Localization/test/00128664.jpg
MIO-TCD-Localization/test/00120951.jpg
MIO-TCD-Localization/train/00009854.jpg
MIO-TCD-Localization/test/00117247.jpg
MIO-TCD-Localization/train/00010540.jpg
MIO-TCD-Localization/train/00042452.jpg
MIO-TCD-Localization/train/00074364.jpg
MIO-TCD-Localization/train/00088140.jpg
MIO-TCD-Localization/test/00129867.jpg
MIO-TCD-Localization/train/00031035.jpg
MIO-TCD-Localization/test/00130553.jpg
MIO-TCD-Localization/train/00086984.jpg
MIO-TCD-Localization/train/00011743.jpg
MIO-TCD-Localization/train/00019456.jpg
MIO-TCD-Localization/train/00043655.jpg
MIO-TCD-Localization/train/00075567.jpg
MIO-TCD-Localization/train/00020142.jpg
MIO-TCD-Localization/train/00008039.jpg
MIO-TCD-Localization/train/00052054.jpg
MIO-TCD-Localization/train/00100267.jpg
MIO-TCD-Localization/test/00131756.jpg
MIO-TCD-Localization/train/00078450.jpg
MIO-TCD-Localization/train/00021345.jpg
MIO-TCD-Localization/train/00096586.

In [4]:
# Riorganizza le cartelle (circa 2 minuti)
shutil.move('./MIO-TCD-Localization/train/', './split/images/')
os.makedirs('./split/labels/')

'./split/images/'

In [8]:
class_to_index = {
    'articulated_truck': 0,
    'bicycle': 1,
    'bus': 2,
    'car': 3,
    'motorcycle': 4,
    'motorized_vehicle': 5,
    'non-motorized_vehicle': 6,
    'pedestrian': 7,
    'pickup_truck': 8,
    'single_unit_truck': 9,
    'work_van': 10
}

In [9]:
# Estrazione di tutte le righe e conversione in singoli label (2:30 min)
csv_file = "MIO-TCD-Localization/gt_train.csv"
images_dir = "split/images/"
output_dir = "split/labels/"
print("Dovrebbero essere 351549 label")

with open(csv_file, 'r') as file:
    reader = csv.reader(file)
    for row in tqdm(reader):
        image_id = row[0]
        class_name = row[1]
        x_min = int(row[2])
        y_min = int(row[3])
        x_max = int(row[4])
        y_max = int(row[5])

        image_path = os.path.join(images_dir, f"{image_id}.jpg")

        # Carica l'immagine per ottenere le dimensioni
        with Image.open(image_path) as img:
            img_width, img_height = img.size

        # Calcola x_centro, y_centro, larghezza e altezza normalizzati
        x_center = (x_min + x_max) / 2.0 / img_width
        y_center = (y_min + y_max) / 2.0 / img_height
        width = (x_max - x_min) / img_width
        height = (y_max - y_min) / img_height

        # Ottieni l'indice della classe
        class_index = class_to_index[class_name]

        # Crea il nome del file .txt
        txt_filename = os.path.join(output_dir, f"{image_id}.txt")

        # Scrivi la linea nel file .txt
        with open(txt_filename, 'a') as txt_file:
            txt_file.write(f"{class_index} {x_center} {y_center} {width} {height}\n")


0it [00:00, ?it/s]

In [10]:
# Splitting casuale (20 secondi)
dataset_path = 'split/'
train_ratio = 0.8

# Percorsi alle sottocartelle
images_dir = os.path.join(dataset_path, 'images')
labels_dir = os.path.join(dataset_path, 'labels')

# Creare sottocartelle per training e validation
train_images_dir = os.path.join(images_dir, 'train')
val_images_dir = os.path.join(images_dir, 'val')
train_labels_dir = os.path.join(labels_dir, 'train')
val_labels_dir = os.path.join(labels_dir, 'val')

Path(train_images_dir).mkdir(parents=True, exist_ok=True)
Path(val_images_dir).mkdir(parents=True, exist_ok=True)
Path(train_labels_dir).mkdir(parents=True, exist_ok=True)
Path(val_labels_dir).mkdir(parents=True, exist_ok=True)

# Ottieni lista di tutti i file
image_files = sorted(os.listdir(images_dir))
label_files = sorted(os.listdir(labels_dir))

# Filtra solo i file immagini e label escludendo le sottocartelle
image_files = [f for f in image_files if os.path.isfile(os.path.join(images_dir, f))]
label_files = [f for f in label_files if os.path.isfile(os.path.join(labels_dir, f))]

# Assicurarsi che ogni immagine abbia una label corrispondente
image_label_pairs = [(img, lbl) for img, lbl in zip(image_files, label_files) if img.rsplit('.', 1)[0] == lbl.rsplit('.', 1)[0]]

# Mescola i dati
random.shuffle(image_label_pairs)

# Calcola il numero di file per il training set
train_size = int(len(image_label_pairs) * train_ratio)

# Divide tra training e validation
train_pairs = image_label_pairs[:train_size]
val_pairs = image_label_pairs[train_size:]

# Copia i file nella cartella corrispondente
print(f"{len(train_pairs)} in train")
for img_file, lbl_file in train_pairs:
    shutil.move(os.path.join(images_dir, img_file), os.path.join(train_images_dir, img_file))
    shutil.move(os.path.join(labels_dir, lbl_file), os.path.join(train_labels_dir, lbl_file))

print(f"{len(val_pairs)} in val")
for img_file, lbl_file in val_pairs:
    shutil.move(os.path.join(images_dir, img_file), os.path.join(val_images_dir, img_file))
    shutil.move(os.path.join(labels_dir, lbl_file), os.path.join(val_labels_dir, lbl_file))


88000 in train
22000 in val


In [11]:
!tar -cvf mio-ds-splitted.tar split/

Output streaming troncato alle ultime 5000 righe.
split/labels/train/00009978.txt
split/labels/train/00042445.txt
split/labels/train/00045451.txt
split/labels/train/00094777.txt
split/labels/train/00009078.txt
split/labels/train/00084889.txt
split/labels/train/00031652.txt
split/labels/train/00018955.txt
split/labels/train/00063503.txt
split/labels/train/00065722.txt
split/labels/train/00035090.txt
split/labels/train/00092887.txt
split/labels/train/00004909.txt
split/labels/train/00018126.txt
split/labels/train/00052501.txt
split/labels/train/00095907.txt
split/labels/train/00035164.txt
split/labels/train/00082456.txt
split/labels/train/00031081.txt
split/labels/train/00075454.txt
split/labels/train/00093774.txt
split/labels/train/00012194.txt
split/labels/train/00108852.txt
split/labels/train/00027163.txt
split/labels/train/00010382.txt
split/labels/train/00105215.txt
split/labels/train/00022425.txt
split/labels/train/00022288.txt
split/labels/train/00099363.txt
split/labels/train/000

In [12]:
shutil.copy('/content/mio-ds-splitted.tar', '/content/drive/MyDrive/Simone/mio_dataset-splitted.tar')

'/content/drive/MyDrive/Simone/mio_dataset-splitted.tar'